In [6]:
# https://developer.spotify.com/console/get-audio-features-several-tracks/?ids=4JpKVNYnVcJ8tuMKjAj50A,2NRANZE9UCmPAS5XVbXL40,24JygzOLM0EmRQeGtFcIcG

#https://www.kaggle.com/aeryan/spotify-music-analysis

#https://www.kaggle.com/cihanoklap/top-songs-on-spotify-what-makes-them-popular

In [9]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [10]:
path = "Charts CSV/regional-global-daily-latest.csv"
spotify_data_global = pd.read_csv(path)
spotify_data_global.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
2,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
3,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
4,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE


In [11]:
spotify_data_global.to_csv("clean_global.csv", header=False, index=False)


In [12]:
path_clean = "clean_global.csv"
spotify_data_global_clean = pd.read_csv(path_clean)
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz


In [26]:
spotify_data_global_clean["ID"] = spotify_data_global_clean["URL"].str[31:]
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL,ID
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e,3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20,5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG,4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE,6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz,3DarAbFujv6eYNliUTyqtz


In [19]:
len("https://open.spotify.com/track/")

31

In [21]:
test = spotify_data_global_clean["URL"].str
print(test[31:])

0      3Wrjm47oTz2sjIgck11l5e
1      5HCyWlXZPP0y6Gqq8TgA20
2      4ZtFanR9U6ndgddUvNcjcG
3      6PQ88X9TkUIAUIZJHW2upE
4      3DarAbFujv6eYNliUTyqtz
                ...          
195    5EVzgDPaM0LCM3M5exl3z8
196    7fBv7CLKzipRk6EC6TWHOB
197    5zqObw7wjBgL9TDiAymxPn
198    3rq5w4bQGigXOfdN30ATJt
199    4JeOesjcWWlprTAvucDSJX
Name: URL, Length: 200, dtype: object


In [ ]:
# token = "BQAEIQPWvNjgZD41CmvBSrkEvOTxVYxWU1l9GaAFb79KuLLanpGlLTccAsdvVz5yK-xzOFVzHnnDYARd-eZbZGKsuQU0Yt8MofmTOiPCW3CRv6PZYmcxHro-BQ25W6gUcbyd6YTxWEccl9-1UP8SNAArj7IjeQ"
# token = "BQC7SbbuUwwebllY8WlsJYjDKwpMeTF8twCsti47Or50rxWXZ0JOYs7IXIrDOb6LnhE5YPUu_nUkkjTma-zh6xPjf_j7AtCIEk4qOdQu8i_wvvjie5Bp1xANNhhhjusybp6spB2ZFhbJ-0dTfVE"
# token = "BQB1hjVmwHas7EQTBD5SgV0fYBJFhkQdLjo6QV9qMxtwM-KWxVOcVt1f1934NQ2wPT4xZ27TQVpUsQV5JG-GR9-GJoE1iAvqR32bHL4ttyRNRgIChRd4sCfUoFeSMhodlYUUv5Tys9sVJk-fdWNXNg"
#token="BQAqFieWd9Vs7_PadOTnr5aa5Z718R_mmO6WACUnALMgjtzORXuE1eV6wxBonSkIlbKt924WCoIdF2Ia9QSgFlKzw2VONVEbgzTYTMUobMQTP4CSybOuLgCyAc8dasFXh8ycBwfmtLKvrHb5D9v7c3t_DxahFixpPZd-TsBPoCbRgw"
token = "BQB7MQS25CAJBY0_A7cLn1k-hjxKiJ9C5efP0m2gS-4FM1O3ySSJjQfZuUktBUKcqv14KmVnLYTmK8tpYLU-KmK9FYUsRl26OzMKrKufqtgCGOaMStQhGA8ykkmM_fBqNdt1LC4kPqj5GoAe7i9GNwcIM57p_guU93xZ1F44eEPvRzQCwgWG_vfWrR0yPHvhPVHxlVWlnvMGuB_al3cpiNWABHRCAI4SNsy15u_CYPXSq6FR047cFkWO_Ijwd5yjeoyz8k4ho5gQL3Oe3geX"
headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.SpotifyTopChartsdb
collection = db.TopGlobal
# collection.drop()

rows=len(spotify_data_global_clean)
global_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    # Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    artist_id = spotify_data_global_clean.iloc[row,5]
    artist_name = spotify_data_global_clean.iloc[row,2]
    song_name = spotify_data_global_clean.iloc[row,1]

    
    response = requests.get(f'https://api.spotify.com/v1/audio-features?ids={artist_id}', headers=headers_dict).json()
    print(response)
    # artist_id = response["audio_features"]
    print(response["audio_features"])

    danceability = response["audio_features"][0]["danceability"]
    energy = response["audio_features"][0]["energy"]
    key = response["audio_features"][0]["liveness"]
    loudness = response["audio_features"][0]["key"]
    acousticness = response["audio_features"][0]["acousticness"]
    instrumentalness = response["audio_features"][0]["instrumentalness"]
    liveness = response["audio_features"][0]["liveness"]
    valence = response["audio_features"][0]["valence"]
    tempo = response["audio_features"][0]["tempo"]
    track_href = response["audio_features"][0]["track_href"]
        
        
    global_dict={'Artist Name':artist_name,
                 'Song Name':song_name,
                'Danceability': danceability,
                 'Energy': energy,
                 'Key': key,
                 'Loudness': loudness,
                 'Acousticness': acousticness,
                 'Instrumentalness': instrumentalness,
                 'Liveness': liveness,
                 'Valence': valence,
                 'Tempo': tempo,
                 'Track Reference': track_href
                 }
    collection.insert_one(global_dict)
#     spotify_artist_dict.append(artist_dict)

{'audio_features': [{'danceability': 0.714, 'energy': 0.8, 'key': 11, 'loudness': -4.808, 'mode': 0, 'speechiness': 0.0504, 'acousticness': 0.127, 'instrumentalness': 0, 'liveness': 0.359, 'valence': 0.589, 'tempo': 134.002, 'type': 'audio_features', 'id': '3Wrjm47oTz2sjIgck11l5e', 'uri': 'spotify:track:3Wrjm47oTz2sjIgck11l5e', 'track_href': 'https://api.spotify.com/v1/tracks/3Wrjm47oTz2sjIgck11l5e', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Wrjm47oTz2sjIgck11l5e', 'duration_ms': 211560, 'time_signature': 4}]}
[{'danceability': 0.714, 'energy': 0.8, 'key': 11, 'loudness': -4.808, 'mode': 0, 'speechiness': 0.0504, 'acousticness': 0.127, 'instrumentalness': 0, 'liveness': 0.359, 'valence': 0.589, 'tempo': 134.002, 'type': 'audio_features', 'id': '3Wrjm47oTz2sjIgck11l5e', 'uri': 'spotify:track:3Wrjm47oTz2sjIgck11l5e', 'track_href': 'https://api.spotify.com/v1/tracks/3Wrjm47oTz2sjIgck11l5e', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Wrjm47oTz2sjIgck11l5e